In [1]:
%matplotlib inline
import pandas as pd
import os
import math
import json
import copy
import numpy as np
import matplotlib.pyplot as plt
import gpflow
import pickle
import calendar
import tensorflow as tf

from datetime import datetime
from gpflow.utilities import print_summary

gpflow.config.set_default_summary_fmt("notebook")

# plotly viz - use matplotlib if you prefer
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# cleanair modules
from cleanair.scoot import (
    sample_n,
    ScootQuery,
    sample_intensity
)


In [2]:
secretfile = "../../terraform/.secrets/db_traffic.json"

# choose a start and end date for this query
start_datetime_LD="2020-03-23 00:00:00"
end_datetime_LD="2020-03-24 00:00:00"

# choose a start and end date for querying "normal traffic" period
start_datetime_normal="2020-02-10 00:00:00"
end_datetime_normal="2020-02-25 00:00:00"

# choose a start and end date for querying "lockdown traffic" period
# start_datetime_LD="2020-03-16 00:00:00"
# end_datetime_LD="2020-03-26 00:00:00"

# columns to analyse
columns = ["n_vehicles_in_interval", "occupancy_percentage", "congestion_percentage", "saturation"]

SQ = ScootQuery(secretfile=secretfile)

2020-03-31 16:57:27     INFO: Database connection information loaded from<_io.TextIOWrapper name='../../terraform/.secrets/db_traffic.json' mode='r' encoding='UTF-8'>


# Detector readings
Get all scoot observations between `start_datetime` and `end_datetime`

In [3]:
all_df = SQ.get_all_readings(start_datetime=start_datetime_normal, end_datetime=end_datetime_normal)

In [10]:
## This is querying the full data which have been already saved - takes around 30 mins
#all_df_normal = SQ.get_all_readings(start_datetime=start_datetime, end_datetime=end_datetime)
#all_df_LD = SQ.get_all_readings(start_datetime=start_datetime_LD, end_datetime=end_datetime_LD)

In [11]:
## Save data - this has been used to save the complete data
all_df_normal.to_csv("normal_scoot.csv")
all_df_LD.to_csv("LD_scoot.csv")

NameError: name 'all_df_normal' is not defined

In [12]:
## Import data - only run if you want to work with the complete dataset
#all_df_normal = pd.read_csv("normal_scoot.csv")
#all_df_LD = pd.read_csv("LD_scoot.csv")

# Simple Data cleaning

    - Convert Datetime to epoch
    - Add normalised/standardised columns

In [4]:
def normalise(x):
    """Standardize all columns individually"""
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)

def denormalise(x, wrt_y):
    """Denormalize x given the original data it was standardized to"""
    return ( x * np.std(wrt_y, axis=0) ) + np.mean(wrt_y, axis=0)

In [5]:
all_df['measurement_start_utc'] = pd.to_datetime(all_df['measurement_start_utc'])
all_df['epoch'] = all_df['measurement_start_utc'].astype('int64')//1e9 #convert to epoch

all_df['epoch_norm'] = normalise(all_df['epoch'])
all_df['lat_norm'] = normalise(all_df['lat'])
all_df['lon_norm'] = normalise(all_df['lon'])

# Helper functions

In [6]:
def get_X(df):
    return np.array(df[['epoch_norm', 'lon_norm', 'lat_norm']])

def get_Y(df):
    return np.array(df[['n_vehicles_in_interval']])

# Jointly train all sensors

The input $X$ is time epoch, lat, lon and output $Y$ is the integer n_vehicles_in_interval

NOTE for 2 days of scoot data there are approx 400000 observations

In [7]:
X = get_X(all_df) # N x D
Y = get_Y(all_df) # N x 1

In [8]:
print(Y.shape)

(3508367, 1)


# Temporary load local dataframe and settings

In [18]:
all_df = pd.read_csv("./data/raw/scoot_data_10feb.csv")
all_df_copy = copy.deepcopy(all_df)

FileNotFoundError: [Errno 2] File b'./data/raw/scoot_data_10feb.csv' does not exist: b'./data/raw/scoot_data_10feb.csv'

In [19]:
all_df = copy.deepcopy(all_df_copy)

NameError: name 'all_df_copy' is not defined

In [9]:
with open('./data/settings/kernel_settings.json') as kernel_file:
    kernel_settings = json.load(kernel_file)
with open('./data/settings/scoot_settings.json') as scoot_file:
    scoot_settings = json.load(scoot_file)

In [10]:
# Specify start and end dates
start_date = '2020-02-10 01:00:00'
end_date = '2020-02-16 23:00:00'

# Train scoot individually as a time series

In [11]:
## Select all scoots in scoot_settings
all_df = all_df[all_df['detector_id'].isin(scoot_settings['scoot_ids'])]

In [12]:
## Select all dates before given date
all_df = all_df[(all_df['measurement_end_utc'] >= start_date) & (all_df['measurement_end_utc'] <= end_date)]

In [13]:
scoot_ids = np.unique(all_df['detector_id'])
grouped_df = all_df.groupby('detector_id')
scoot_individual_df_arr = [grouped_df.get_group(scoot_id) for scoot_id in scoot_ids] #array of dfs for all sensors

In [14]:
# scoot_individual_df_arr = [grouped_df.get_group(scoot_id) for scoot_id in scoot_ids] #array of dfs for all sensors
X_arr = [get_X(all_df) for df in scoot_individual_df_arr] # |Number of scoot sensors| x N_i x D
Y_arr = [get_Y(all_df) for df in scoot_individual_df_arr] # |Number of scoot sensors| x N_i x 1

In [15]:
len(X_arr)

2

## Simple Time series plot

In [16]:
color_counts = 'C0'
label_counts = 'N'
color_estimated_counts = 'red'
label_estimated_counts = '$\hat{N}$'

In [17]:
scoot_id = 0
sensor_df = scoot_individual_df_arr[scoot_id]

fig = go.Figure()

fig.add_trace(go.Scatter(x=sensor_df['measurement_start_utc'], y=sensor_df['n_vehicles_in_interval'],
                    mode='lines+markers',
                    name='lines+markers')
)

fig.update_layout(title='Timeseries of sensor {scoot_id}'.format(scoot_id=scoot_ids[scoot_id]),
                xaxis_title="Datetime",
                yaxis_title="# of vechicles per hour",
                font=dict(
                    size=16)
)

fig.show()

# Fit LGCP model for each sensor

In [18]:
## Set random seed
gpflow.config.set_default_float(np.float64)
np.random.seed(0)
tf.random.set_seed(0)

NameError: name 'gpflow' is not defined

In [30]:



## Optimization functions - train the model for the given epochs
optimizer = tf.keras.optimizers.Adam(0.001)
def optimization_step(model: gpflow.models.SVGP, X, Y):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        obj = -model.elbo(X, Y)
        grads = tape.gradient(obj, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
def simple_training_loop(X, Y, model: gpflow.models.SVGP, epochs: int = 1, logging_epoch_freq: int = 10, num_batches_per_epoch: int = 10):
    tf_optimization_step = tf.function(optimization_step)
    for epoch in range(epochs):
        for _ in range(num_batches_per_epoch):
            tf_optimization_step(model, X, Y)

        epoch_id = epoch + 1
        if epoch_id % logging_epoch_freq == 0:
            tf.print(f"Epoch {epoch_id}: ELBO (train) {model.elbo(X,Y)}")
            

NameError: name 'tf' is not defined

In [31]:
## Given the data and the specific sensor this function optimise the ELBO and plot the results 
def train_sensor_model(scoot_id, X_arr, Y_arr, kernelsettings, epochs = 100, logging_epoch_freq = 10, plot=True):
    
    ## To remove newaxis when more features
    num_features = X_arr[scoot_id][:,0][:,np.newaxis].shape[0]
    
    X = tf.convert_to_tensor(X_arr[scoot_id][:,0][:,np.newaxis])
    Y = tf.convert_to_tensor(Y_arr[scoot_id].astype(np.float64))
    
    ## To pass it as a function arg
    k = choose_kernel(kernelsettings)
#     k = gpflow.kernels.RBF() * gpflow.kernels.Periodic(0.1)
    
    lik = gpflow.likelihoods.Poisson()
    
    ## Add code for inducing inputs - Needed when we run on the full data
    model = gpflow.models.SVGP(kernel = k, likelihood=lik, inducing_variable=X)
    
    ## Uncomment to see which variables are training and those that are not
    #print_summary(model)
    
    simple_training_loop(X, Y, model, epochs = epochs, 
                         logging_epoch_freq = logging_epoch_freq)
    
    if plot == True:
#         plot_results(scoot_individual_df_arr, sensor_df, model, X)
        plotly_results(scoot_id, 2, model, X)
    return model,X

In [32]:
# THIS IS BUGGY
## Computes percentage cover (see Virginia's pdf for details)
def percentage_coverage(model,test_inputs,Ytest,quantile:int = 0.95, num_samples:int = 10,num_pertubations: int = 100):
    # Number of times total counts were within 90th percentile
    coverage_events = 0
    
    # Loop over pertubations
    for i in range(num_pertubations):

        # Change seed
        np.random.seed(i)
        
        # Sample from latent function (intensity)
        intensity_sample = np.exp(model.predict_f_samples(test_inputs,num_samples))
        # Compute emprical distribution of counts
        empirical_count_distribution = np.random.poisson(intensity_sample)
        
        # Total number of actual counts
        total_counts = np.sum(Ytest)
       
        # Compute upper and lower quantiles from the empirical distribution of counts
        upper_q = np.quantile(np.sum(samples[:,:,0],axis=1),quantile)
        lower_q = np.quantile(np.sum(samples[:,:,0],axis=1),1-quantile)
    
        # Add 1 - if total counts are within quantile, 0 - otherwise
        coverage_events += int((total_counts < upper_q) & (total_counts > lower_q))
        binary = int((total_counts < upper_q) & (total_counts > lower_q)) # this is kept for debugging (remove afterwards)

    return empirical_count_distribution, binary, total_counts, upper_q, lower_q # this is kept for debugging (remove afterwards)
    return coverage_events/num_pertubations # this should be the output after debugging



# Run entire training routine with plotting

In [33]:
epochs = 1000
logging_epoch_freq = 100

In [34]:
model0,Xtest0 = train_sensor_model(0, X_arr, Y_arr, kernel_settings, epochs, logging_epoch_freq)

NameError: name 'X_arr' is not defined

In [35]:
# model1,Xtest1 = train_sensor_model(1, X_arr, Y_arr, kernel_settings, epochs, logging_epoch_freq)

# Save results

In [36]:
save_model_and_metadata(scoot_ids[scoot_id],model0,X_arr[scoot_id],Y_arr[scoot_id],start_date,end_date,kernel_settings,scoot_settings)



NameError: name 'save_model_and_metadata' is not defined

In [37]:
model0,X_arr,Y_arr,metadata = load_model_and_metadata('N00/002e1','10Feb_16Feb')



NameError: name 'load_model_and_metadata' is not defined

In [38]:
plotly_results(0, 2, model0, X_arr, 1000)

NameError: name 'plotly_results' is not defined